In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!! pip install -U open-metric-learning

['Requirement already satisfied: open-metric-learning in /usr/local/lib/python3.10/dist-packages (2.1.6)',
 'Requirement already satisfied: torch<=2.1.1,>=2.0 in /usr/local/lib/python3.10/dist-packages (from open-metric-learning) (2.1.0+cu121)',
 'Requirement already satisfied: pytorch-lightning<=2.1.2,>=2.0 in /usr/local/lib/python3.10/dist-packages (from open-metric-learning) (2.1.2)',
 'Requirement already satisfied: torchvision<=0.16.1 in /usr/local/lib/python3.10/dist-packages (from open-metric-learning) (0.16.0+cu121)',
 'Requirement already satisfied: albumentations<=1.3.1,>=1.0 in /usr/local/lib/python3.10/dist-packages (from open-metric-learning) (1.3.1)',
 'Requirement already satisfied: opencv-python>=4.1.1 in /usr/local/lib/python3.10/dist-packages (from open-metric-learning) (4.8.0.76)',
 'Requirement already satisfied: pillow>=8.2 in /usr/local/lib/python3.10/dist-packages (from open-metric-learning) (9.4.0)',
 'Requirement already satisfied: pandas>=1.0 in /usr/local/lib

In [ ]:
import torch
from tqdm import tqdm

from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_all_tri import AllTripletsMiner
from oml.models import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.utils.download_mock_dataset import download_mock_dataset

In [ ]:
import os
os.chdir("/content/drive/MyDrive/2024_공부/OML/img/WOMAN")

# path
fpath = os.listdir()
path = sorted(fpath)

# label
n = 0
label = []
for i in path:
  fst = [word[0] for word in path]
  for letter in fst:
    if letter == fst[0]:
      label.append(n)
    else:
      break
  n += 1

  if len(label) >= len(path):
    break

In [ ]:
# split
split = [0 for _ in label]
idx = int(len(label) * 0.7)

if label[idx] == label[idx - 1] and label[idx] == label[idx + 1]:
  idx -= 1

elif label[idx] == label[idx + 1]:
  idx += 1

for i in range(idx,len(split)):
  split[i] = 1

In [ ]:
# is_query, is_gallery
import numpy as np
is_query = ['NaN' for _ in label]
is_gallery = ['NaN' for _ in label]

for i in range(idx,len(label)):
  if label[i] != label[i - 1] and label[i] == label[i + 1]:
    is_query[i] = 'True'
    is_gallery[i] = 'False'
  else:
    is_query[i] = 'False'
    is_gallery[i] = 'True'

In [ ]:
# 테이블 생성
import os
os.chdir("/content/drive/MyDrive/2024_공부/OML/")
import sqlite3

conn = sqlite3.connect('img/w_img.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE w_img
                  (label, path, split, is_query, is_gallery)''')

In [ ]:
# db에 데이터 삽입
for i in range(len(label)):
    cursor.execute("insert into w_img values (?, ?, ?, ?, ?)", (label[i], path[i], split[i], is_query[i], is_gallery[i]))

conn.commit()

In [ ]:
# DF 타입 출력
import pandas as pd

cursor.execute("SELECT * FROM w_img;")
rows = cursor.fetchall()

cols = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(data=rows, columns=cols)
conn.close()
df

,label,path,split,is_query,is_gallery
0,0,Anyujin01.jpeg,0,NaN,NaN
1,0,Anyujin02.jpeg,0,NaN,NaN
2,0,Anyujin03.jpeg,0,NaN,NaN
3,1,Goyunjeong01.jpeg,0,NaN,NaN
4,1,Goyunjeong02.jpeg,0,NaN,NaN
5,1,Goyunjeong03.jpeg,0,NaN,NaN
6,2,Hansohui01.jpg,0,NaN,NaN
7,2,Hansohui02.jpg,0,NaN,NaN
8,2,Hansohui03.jpg,0,NaN,NaN
9,3,Heoyunjin01.jpg,0,NaN,NaN


In [ ]:
import pandas as pd

df_train = df[df['split'] == 0]
dataset_root = "img/WOMAN/"

In [ ]:
import cv2
import torch
from torchvision import transforms

# 이미지를 읽고, 크기를 조정하고, 텐서로 변환하는 함수
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)  # 이미지 읽기
    if image is None:
        print(f"이미지를 읽을 수 없습니다: {image_path}")
        return None

    # 이미지 크기 조정
    resized_image = cv2.resize(image, target_size)

    # 이미지를 RGB로 변환
    resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

    # 이미지를 텐서로 변환
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    tensor_image = transform(resized_image)

    return tensor_image

# 모든 이미지를 동일한 크기로 조정하여 텐서로 변환
tensor_images = []
image_paths = dataset_root + df["path"]
for img_path in image_paths:
    tensor_img = preprocess_image(img_path)
    if tensor_img is not None:
        tensor_images.append(tensor_img)

In [ ]:
import cv2

for img_path in image_paths:
    image = cv2.imread(img_path)
    print(f"이미지 경로: {img_path}, 이미지 크기: {image.shape}")

이미지 경로: img/WOMAN/Anyujin01.jpeg, 이미지 크기: (1599, 1066, 3)
이미지 경로: img/WOMAN/Anyujin02.jpeg, 이미지 크기: (2048, 1536, 3)
이미지 경로: img/WOMAN/Anyujin03.jpeg, 이미지 크기: (2048, 1366, 3)
이미지 경로: img/WOMAN/Goyunjeong01.jpeg, 이미지 크기: (1633, 1440, 3)
이미지 경로: img/WOMAN/Goyunjeong02.jpeg, 이미지 크기: (647, 647, 3)
이미지 경로: img/WOMAN/Goyunjeong03.jpeg, 이미지 크기: (651, 1179, 3)
이미지 경로: img/WOMAN/Hansohui01.jpg, 이미지 크기: (900, 600, 3)
이미지 경로: img/WOMAN/Hansohui02.jpg, 이미지 크기: (750, 500, 3)
이미지 경로: img/WOMAN/Hansohui03.jpg, 이미지 크기: (503, 720, 3)
이미지 경로: img/WOMAN/Heoyunjin01.jpg, 이미지 크기: (767, 540, 3)
이미지 경로: img/WOMAN/Heoyunjin02.jpg, 이미지 크기: (877, 719, 3)
이미지 경로: img/WOMAN/Heoyunjin03.jpg, 이미지 크기: (849, 680, 3)
이미지 경로: img/WOMAN/Jisu01.jpg, 이미지 크기: (956, 680, 3)
이미지 경로: img/WOMAN/Jisu02.jpg, 이미지 크기: (854, 610, 3)
이미지 경로: img/WOMAN/Jisu03.jpg, 이미지 크기: (874, 700, 3)
이미지 경로: img/WOMAN/Karina01.jpg, 이미지 크기: (832, 436, 3)
이미지 경로: img/WOMAN/Karina02.jpg, 이미지 크기: (788, 666, 3)
이미지 경로: img/WOMAN/Karina03.jpg, 이미지 크기: (49

In [ ]:
image_paths = dataset_root + df["path"]

In [ ]:
model = ViTExtractor("vits16_dino", arch="vits16", normalise_features=False).train()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
train_dataset = DatasetWithLabels(df_train, dataset_root=dataset_root)
criterion = TripletLossWithMiner(margin=0.1, miner=AllTripletsMiner(), need_logs=True)
sampler = BalanceSampler(train_dataset.get_labels(), n_labels=2, n_instances=2)

# torch.utils.data.DataLoader(): 데이터를 미니배치 형태로 만듦
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=sampler)

https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth
Checkpoint is already here.


In [ ]:
for batch in tqdm(train_loader):
    # print(batch["input_tensors"])
    embeddings = model(batch["input_tensors"])

    loss = criterion(embeddings, batch["labels"])
    print("\n",batch["input_tensors"], "\n", batch["labels"])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # info for logging: positive/negative distances, number of active triplets
    print(criterion.last_logs)

  0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [3, 647, 647] at entry 0 and [3, 651, 1179] at entry 1